In [ ]:
import numpy as np 

from keras.datasets import mnist 
from keras.utils import to_categorical

from dense import Dense 
from convolution import Convolutional 
from reshape import Reshape 
from activations import Sigmoid
from maxpooling import MaxPooling2D
from losses import binary_cross_entropy, binary_cross_entropy_prime

In [ ]:
def preprocess_data(x, y, limit):
    # take only y = 0 and y = 1, make mnist digit dataset binary 
    zero_index = np.where(y == 0)[0][:limit]
    one_index  = np.where(y == 1)[0][:limit]
    all_indices= np.hstack((zero_index, one_index))
    all_indices= np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype('float32') / 255 
    y = to_categorical(y) 
    y = y.reshape(len(y), 2, 1) 
    return x, y 

(x_train, y_train), (x_test, y_test) = mnist.load_data() 
x_train, y_train= preprocess_data(x_train, y_train, 100) 
x_test, y_test  = preprocess_data(x_test, y_test, 100)

In [ ]:
# create network! 
network = [
    Convolutional((1, 28, 28), kernel_size=4, n_filters=5, padding=1, stride=2),
    Sigmoid(), 
    MaxPooling2D(),
    Reshape((5, 7, 7), (5 * 7 * 7, 1)),
    Dense(5 * 7 * 7, 100), 
    Sigmoid(), 
    Dense(100, 2), 
    Sigmoid() 
]

In [ ]:
epochs = 10
learning_rate = 0.1 

# train model 
def train(x, y, network, loss, prime_loss, epoch, l_rate):
    for e in range(epochs): 
        error = 0 
        for x_sample, y_sample in zip(x, y):
        # forward 
            output = x_sample 
            for layer in network:           
                output = layer.forward(output)
    
            # error 
            error += loss(y_sample, output) 

            # backward
            grad = prime_loss(y_sample, output)
            for layer in reversed(network):
                grad = layer.backward(grad, l_rate)
        error /= len(x_train) 
        print(f'{e + 1}/{epochs}, error={round(error, 2)}')
    
    return error

In [ ]:
# test model
def test(x, y, network):
    correct = 0
    total   = len(x)
    
    for x_sample, y_sample in zip(x, y): 
        output = x_sample 
        for layer in network: 
            output = layer.forward(output)
        
        pred = np.argmax(output)
        true = np.argmax(y_sample) 
        
        print(f'Pred: {pred}, True: {true}')
        
        if pred == true: 
            correct += 1
            
    accuracy = correct / total 
    print(f'Test Accuracy: {accuracy:.2%}')

In [6]:
train_error = train(x_train, y_train, network, binary_cross_entropy, binary_cross_entropy_prime, epochs, learning_rate)

1/10, error=0.31
2/10, error=0.06
3/10, error=0.03
4/10, error=0.02
5/10, error=0.02
6/10, error=0.01
7/10, error=0.01
8/10, error=0.01
9/10, error=0.01
10/10, error=0.01


In [7]:
test(x_test, y_test, network) 

Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 0, True: 0
Pred: 0, True: 0
Pred: 1, True: 1
Pred: 1, True:

In [ ]:
# the code above was modified from https://github.com/TheIndependentCode/Neural-Network/tree/master